In [1]:
import numpy as np
import pandas as pd

from scipy.stats import randint
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split 
from sklearn.preprocessing import MaxAbsScaler, OrdinalEncoder
from sklearn.tree import DecisionTreeRegressor 

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Table of Contents

[Introduction](#1)

[Data Description](#2)

[Data Preparation](#3)

- [Checking Duplicates, Data Types, and Column Names](#3.1)
- [Exploratory Data Analysis](#3.2)
- [Encoding the Dataframe](#3.3)

[Model Training](#4)
- [Linear Regression Model](#4.1)
- [Decision Tree Model, No Hyperparameter Tuning](#4.2)
- [Decision Tree Model, With Hyperparameter Tuning](#4.3)
- [Random Forest Model, No Hyperparameter Tuning](#4.4)
- [Random Forest Model, With Hyperparameter Tuning](#4.5)
- [CatBoost Regressor, No Hyperparameter Tuning](#4.6)
- [CatBoost Regressor, With Hyperparameter Tuning](#4.7)
- [LightGBM Regressor, No Hyperparameter Tuning](#4.8)
- [LightGBM Regressor, With Hyperparameter Tuning](#4.9)
- [XGB Regressor, No Hyperparameter Tuning](#4.10)
- [XGB Regressor, With Hyperparameter Tuning](#4.11)

[Model Analysis](#5)

[Conclusion](#6)

# Introduction <a id=1></a>

Rusty Bargain is a used car sales company that is developing an app to attract new customers. 

The purpose of the app is to quickly determine the market value of a customer's used car, and it uses historical data about cars (specifically, the specifications, the trim versions, and the prices) to accomplish this. 

My responsibility is to build the model that the app uses to determine the value of a given used car.

Rusty Bargain is interested in the following aspects of my model:

- The accuracy of the predicted prices 
- The speed at which the model makes its predictions
- The time required for training the model

To help me build a sufficiently good model, I will do the following:

(1) Download and look at the given dataset to determine if any preprocessing is needed.

(2) Train different machine learning models with various hyperparameters. Specifically, I will compare gradient boosting methods (LightGBM, CatBoost, and XGBoost) with decision tree, random forest, and linear regression models.

(3) Analyze both the speed and the quality of the models.

The following are some additional notes about how this project will be conducted:
- The RMSE (root mean square error) metric will be used to evaluate the models.
- Linear regression is not very good for hyperparameter tuning, but it is perfect for doing a sanity check of other methods.
- I will use the **%%time** command to find the execution time of my models.

# Data Description <a id=2></a>

The dataset, as it is given to me, has the following features and target.

**Features**

DateCrawled — The date the profile of the vehicle was downloaded from the database

VehicleType — The vehicle's body type

RegistrationYear — The vehicle's registration year

Gearbox — The gearbox type of the vehicle

Power — The horsepower of the vehicle

Model — The vehicle's model

Mileage — The vehicle's mileage (in kilometers)

RegistrationMonth — The vehicle's registration month

FuelType — The vehicle's fuel type

Brand — The vehicle's brand

NotRepaired — Has the vehicle been repaired or not

DateCreated — The date of the vehicle's profile creation

NumberOfPictures — The number of pictures of the vehicle

PostalCode — The postal code of the vehicle's profile's owner

LastSeen — The date of the last activity of the user

**Target**

Price — The price of the used vehicle in euros

# Data Preparation <a id=3></a>

In [2]:
main_df = pd.read_csv('/datasets/car_data.csv')

I wonder if there are any rows in the dataframe that are exact duplicates. 

If so, then they should be dropped because it is incredibly unlikely that two or more distinct vehicles would have the exact same values.

**Checking Duplicates, Data Types, and Column Names** <a id=3.1></a>

In [3]:
main_df[main_df.duplicated()]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
14266,21/03/2016 19:06,5999,small,2009,manual,80,polo,125000,5,petrol,volkswagen,no,21/03/2016 00:00,0,65529,05/04/2016 20:47
27568,23/03/2016 10:38,12200,bus,2011,manual,125,zafira,40000,10,gasoline,opel,no,23/03/2016 00:00,0,26629,05/04/2016 07:44
31599,03/04/2016 20:41,4950,wagon,2003,auto,170,e_klasse,150000,4,gasoline,mercedes_benz,no,03/04/2016 00:00,0,48432,05/04/2016 21:17
33138,07/03/2016 20:45,10900,convertible,2005,auto,163,clk,125000,5,petrol,mercedes_benz,no,07/03/2016 00:00,0,61200,21/03/2016 03:45
43656,13/03/2016 20:48,4200,sedan,2003,manual,105,golf,150000,10,gasoline,volkswagen,no,13/03/2016 00:00,0,14482,13/03/2016 20:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349709,03/04/2016 20:52,700,small,1999,manual,60,ibiza,150000,12,petrol,seat,yes,03/04/2016 00:00,0,6268,05/04/2016 21:47
351555,26/03/2016 16:54,3150,bus,2003,manual,86,transit,150000,11,gasoline,ford,no,26/03/2016 00:00,0,96148,02/04/2016 07:47
352384,15/03/2016 21:54,5900,wagon,2006,manual,129,3er,150000,12,petrol,bmw,no,15/03/2016 00:00,0,92526,20/03/2016 21:17
353057,05/03/2016 14:16,9500,small,2013,manual,105,ibiza,40000,5,petrol,seat,no,04/03/2016 00:00,0,61381,05/04/2016 19:18


Wow... I don't know how the dataframe ended up having 262 duplicated rows, but whatever the reason, the next line drops them.

In [4]:
main_df = main_df.drop_duplicates().reset_index(drop = True)

In [5]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354107 entries, 0 to 354106
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354107 non-null  object
 1   Price              354107 non-null  int64 
 2   VehicleType        316623 non-null  object
 3   RegistrationYear   354107 non-null  int64 
 4   Gearbox            334277 non-null  object
 5   Power              354107 non-null  int64 
 6   Model              334406 non-null  object
 7   Mileage            354107 non-null  int64 
 8   RegistrationMonth  354107 non-null  int64 
 9   FuelType           321218 non-null  object
 10  Brand              354107 non-null  object
 11  NotRepaired        282962 non-null  object
 12  DateCreated        354107 non-null  object
 13  NumberOfPictures   354107 non-null  int64 
 14  PostalCode         354107 non-null  int64 
 15  LastSeen           354107 non-null  object
dtypes: int64(7), object(

Most of the Dtypes seems appropriate for their corresponding columns, which is great! The only exceptions are **DateCrawled**, **DateCreated**, and **LastSeen**, which are better off having Dtype **datetime64** instead of merely **object**.

In [6]:
main_df['DateCrawled'] = pd.to_datetime(main_df['DateCrawled'], format = '%d/%m/%Y %H:%M')
main_df['DateCreated'] = pd.to_datetime(main_df['DateCreated'], format = '%d/%m/%Y %H:%M')
main_df['LastSeen'] = pd.to_datetime(main_df['LastSeen'], format = '%d/%m/%Y %H:%M')

One problem I noticed is that the column names are not in snake_case. The next line converts them into snake_case because this format is standard practice.

In [7]:
main_df = main_df.rename(columns = {'DateCrawled': 'date_crawled', 
                                    'Price': 'price',
                                    'VehicleType': 'vehicle_type',
                                    'RegistrationYear': 'registration_year',
                                    'Gearbox': 'gear_box',
                                    'Power': 'horsepower',
                                    'Model': 'model',
                                    'Mileage': 'mileage',
                                    'RegistrationMonth': 'registration_month',
                                    'FuelType': 'fuel_type',
                                    'Brand': 'brand',
                                    'NotRepaired': 'not_repaired',
                                    'DateCreated': 'date_created',
                                    'NumberOfPictures': 'number_of_pictures',
                                    'PostalCode': 'postal_code',
                                    'LastSeen': 'last_seen'})

Notice that I not only converted the column name **Power** to snake_case, I also changed its name to **horsepower** to be more specific.

Since there are so many columns, I think it is a good idea to rearrange the columns to be in alphabetical order.

In [8]:
main_df = main_df.sort_index(axis = 1)

The only exception I will make to the columns being in alphabetical order is that I would like for the **price** column to be last, since it seems logical for the target to appear last in the dataframe.

In [9]:
price_data = main_df['price'].copy()

main_df = main_df.drop('price', axis = 1)

main_df['price'] = price_data

When working on any project, it is common for me to display a random sample of rows so that I know what the dataset "looks like".

In [10]:
main_df.sample(10)

,brand,date_crawled,date_created,fuel_type,gear_box,horsepower,last_seen,mileage,model,not_repaired,number_of_pictures,postal_code,registration_month,registration_year,vehicle_type,price
106919,audi,2016-03-06 01:57:00,2016-03-06,petrol,manual,125,2016-03-06 08:38:00,150000,a6,no,0,89079,10,1998,sedan,1200
278981,bmw,2016-03-15 21:47:00,2016-03-15,petrol,manual,150,2016-03-17 18:47:00,150000,3er,no,0,59889,11,1998,sedan,3199
6923,ford,2016-03-30 11:48:00,2016-03-30,gasoline,manual,143,2016-04-07 02:44:00,150000,s_max,no,0,51109,4,2008,bus,9399
142218,toyota,2016-03-20 13:53:00,2016-03-20,petrol,manual,87,2016-03-25 13:45:00,90000,yaris,no,0,51149,2,2008,small,4350
125307,bmw,2016-04-04 16:37:00,2016-04-04,gasoline,manual,150,2016-04-04 16:37:00,150000,3er,no,0,39116,12,2004,sedan,3999
285899,mercedes_benz,2016-03-28 20:42:00,2016-03-28,petrol,auto,170,2016-04-07 02:47:00,125000,e_klasse,no,0,40667,7,1999,sedan,3650
121459,volkswagen,2016-03-26 15:42:00,2016-03-26,petrol,manual,75,2016-04-06 03:17:00,150000,golf,NaN,0,29525,6,1994,convertible,2200
206832,bmw,2016-03-08 19:53:00,2016-03-08,petrol,auto,171,2016-03-09 11:45:00,150000,NaN,no,0,28832,3,1998,sedan,1900
178542,mercedes_benz,2016-03-15 00:57:00,2016-03-15,gasoline,auto,143,2016-03-27 18:44:00,150000,e_klasse,NaN,0,99974,1,2002,wagon,2100
311256,renault,2016-03-22 16:56:00,2016-03-22,petrol,manual,95,2016-03-22 17:40:00,150000,megane,no,0,47877,12,2001,wagon,900


**Exploratory Data Analysis** <a id=3.2></a>

Now let's look at each column, except the date and postal code columns, to see if there are any unusual values. For the more quantitative features, I use the **describe** function, whereas for the more categorical features I use **value_counts()**.

The reason why the three date columns will not be analyzed is because the purpose of the app is to quickly determine the market value of a customer's used car, and at the time when the customer is entering the details about their car into the app, the **DateCrawled**, **DateCreated**, and **LastSeen** values would simply not exist yet. (Actually, maybe the **LastSeen** value might exist if the customer is a returner, but it certainly would not exist if the customer is new.)

For the same reason, the three date columns will not be used to train the models.

As for postal code, where the customer lives should not affect the price of the car. In fact, the next line shows that there is virtually 0 correlation between the postal code values and the prices. For this reason, I will not use the **postal_code** column to train the models. 

In [11]:
print('The correlation between the postal code of a customer and the price of the car of the customer is about',
     round(main_df['postal_code'].corr(main_df['price']), 2))

The correlation between the postal code of a customer and the price of the car of the customer is about 0.08


In [12]:
main_df['brand'].value_counts().sort_index()

alfa_romeo         2311
audi              29439
bmw               36881
chevrolet          1751
chrysler           1439
citroen            5143
dacia               898
daewoo              542
daihatsu            806
fiat               9634
ford              25163
honda              2817
hyundai            3583
jaguar              505
jeep                677
kia                2463
lada                225
lancia              471
land_rover          545
mazda              5611
mercedes_benz     32025
mini               3201
mitsubishi         3022
nissan             4936
opel              39902
peugeot           10988
porsche             758
renault           17915
rover               486
saab                526
seat               6901
skoda              5490
smart              5241
sonstige_autos     3373
subaru              762
suzuki             2320
toyota             4601
trabant             589
volkswagen        76960
volvo              3207
Name: brand, dtype: int64

Nothing unusual with this column!

In [13]:
main_df['fuel_type'].value_counts()

petrol      216161
gasoline     98658
lpg           5307
cng            565
hybrid         233
other          204
electric        90
Name: fuel_type, dtype: int64

Nothing unusual with their column either! However, I did notice that this is one of the columns that has null values. It would be very difficult, if not impossible, to figure out which fuel type each null value is meant to be, so instead I will fill the null values with the word "unknown".

In [14]:
main_df['fuel_type'] = main_df['fuel_type'].fillna('unknown')

In [15]:
main_df['gear_box'].value_counts()

manual    268034
auto       66243
Name: gear_box, dtype: int64

It makes sense that there are far more vehicles with manual gear boxes than automatic because it is only in recent years that automatic gear boxes have become more common than manual in some parts of the world.

I noticed that this is one of the columns that has null values. It would be very difficult, if not impossible, to figure out which of the two gear boxes each null value is meant to be, so instead I will fill the null values with the word "unknown". 

In [16]:
main_df['gear_box'] = main_df['gear_box'].fillna('unknown')

In [17]:
main_df['horsepower'].describe()

count    354107.000000
mean        110.089651
std         189.914972
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: horsepower, dtype: float64

This column has both unusually small and unusually large horsepower values. There is at least one car in the dataframe with a horsepower of 0, which is impossible, and there is at least one car with a horsepower of 20,000 which is also impossible.

When Henry Ford introduced the Ford Model T in 1908, even those had a horsepower value of at least 20. On the flip side, I think any car whose apparent horsepower value is greater than 500 should be viewed as suspicious because even 500 is a rare horsepower value, a value one would probably only see in high-end vehicles.

How many cars have "suspicious" horsepower values? Let's find out!

In [18]:
print('The dataframe has', len(main_df[main_df['horsepower'] < 20]), 
      'cars that apparently have a horsepower less than 20.')

print('The dataframe has', len(main_df[main_df['horsepower'] > 500]), 
      'cars that apparently have a horsepower greater than 500.')

The dataframe has 40635 cars that apparently have a horsepower less than 20.
The dataframe has 459 cars that apparently have a horsepower greater than 500.


Wow, there are a lot of cars that have "suspicious" horsepower values (especially suspiciously low horsepowers)!

I do not want to drop these rows because 40,635 + 459 = 41,094 is a lot of data. Instead, I will group the dataframe by **brand**, **model**, and **vehicle_type** and find the median horsepower value from grouping this way.

I will also apply a lambda function that replaces the values less than 20 or greater than 500 with the corresponding median values.

However, before I do this I must address the fact that the **model** and **vehicle_type** columns have null values. I could drop the rows with a null **model** and/or **vehicle_type** value, but that would risk losing a lot of data. Instead, I will fill these null values with the word "unknown".

In [19]:
main_df['model'] = main_df['model'].fillna('unknown')
main_df['vehicle_type'] = main_df['vehicle_type'].fillna('unknown')

Now let's use **groupby** and the lambda function.

In [20]:
median_hp = main_df.groupby(['brand', 'model', 'vehicle_type'])['horsepower'].median().reset_index()

main_df = pd.merge(main_df, median_hp, on = ['brand', 'model', 'vehicle_type'], suffixes = ('', '_median'))

main_df['horsepower'] = main_df.apply(lambda row: row['horsepower_median'] 
                                       if ((row['horsepower'] < 20) or (row['horsepower'] > 500))
                                       else row['horsepower'], axis = 1)

main_df = main_df.drop(columns = ['horsepower_median'])

In [21]:
main_df['horsepower'].describe()

count    354107.000000
mean        116.533983
std          55.045584
min           0.000000
25%          75.000000
50%         109.000000
75%         143.000000
max        5000.000000
Name: horsepower, dtype: float64

The lambda function definitely changed the summary statistics of the **horsepower** column, but somehow there is still at least one car with a horsepower value less than 20 (specifically, 0) and at least one car with a horsepower value more than 500 (specifically, 5000). This is likely because certain "groups" of cars, unfortunately, have unusually low or unusually high median horsepower values. How many cars still have "suspicious" horsepower values?

In [22]:
print('The dataframe (still) has', len(main_df[main_df['horsepower'] < 20]), 
      'cars that apparently have a horsepower less than 20.')

print('The dataframe (still) has', len(main_df[main_df['horsepower'] > 500]), 
      'cars that apparently have a horsepower greater than 500.')

The dataframe (still) has 3656 cars that apparently have a horsepower less than 20.
The dataframe (still) has 7 cars that apparently have a horsepower greater than 500.


I am surprised that there are still 3,656 + 7 = 3,663 rows with such horsepower values, but thankfully this is far fewer than the 41,094 I had before. So much fewer to the point that I think it is reasonable to drop these 3,663 rows from the dataframe entirely.

In [23]:
main_df = main_df[(main_df['horsepower'] >= 20) & (main_df['horsepower'] <= 500)]
main_df['horsepower'].describe()

count    350444.000000
mean        117.699363
std          52.640634
min          20.000000
25%          75.000000
50%         109.000000
75%         143.000000
max         500.000000
Name: horsepower, dtype: float64

In [24]:
main_df['mileage'].describe()

count    350444.000000
mean     128571.954435
std       37409.230260
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: mileage, dtype: float64

These values make sense. One can expect that the majority of used cars being sold have a high mileage, but it also makes sense for a small percentage of them to have low mileage, e.g. the one(s) with the minimum mileage value of 5,000. The low values make sense because some people deliberately use their high-value used cars gently in order to sell them for a high price.

In [25]:
main_df['model'].value_counts()

golf                  29215
other                 23832
3er                   19744
unknown               17625
polo                  13057
                      ...  
serie_2                   6
kalina                    6
rangerover                3
serie_3                   3
range_rover_evoque        2
Name: model, Length: 250, dtype: int64

This column does not appear to have anything unusual about it. Intuitively, it makes sense that some models are far more common than others because it is not unheard of for car companies to release limited edition models.

In [26]:
main_df['not_repaired'].value_counts()

no     246243
yes     35808
Name: not_repaired, dtype: int64

In my opinion, it is very awkward (and possibly misleading) for a column name to use the word "not". I will rename this column to **is_repaired**, and make the **yes** and **no** values trade places.

In [27]:
main_df = main_df.rename(columns = {'not_repaired': 'is_repaired'})

main_df['is_repaired'] = main_df['is_repaired'].replace('yes', 'change_to_no')
main_df['is_repaired'] = main_df['is_repaired'].replace('no', 'change_to_yes')

main_df['is_repaired'] = main_df['is_repaired'].replace('change_to_yes', 'yes')
main_df['is_repaired'] = main_df['is_repaired'].replace('change_to_no', 'no')

Also, this column has null values, which I will now replace with the word "unknown".

In [28]:
main_df['is_repaired'] = main_df['is_repaired'].fillna('unknown')

In [29]:
main_df['number_of_pictures'].value_counts()

0    350444
Name: number_of_pictures, dtype: int64

This is bizarre... I honestly have no idea how it can be that none of the 350,444 cars have any pictures. Whatever the reason, this column is completely useless for my models because if all of the values are 0, then the models cannot use the values of this column to make predictions about the value of a particular used car. The next line drops this column from the dataframe.

In [30]:
main_df = main_df.drop('number_of_pictures', axis = 1)

In [31]:
main_df['price'].describe()

count    350444.000000
mean       4442.910234
std        4519.722739
min           0.000000
25%        1100.000000
50%        2750.000000
75%        6480.000000
max       20000.000000
Name: price, dtype: float64

There is at least one used car in the dataframe that, for one reason or another, is worth 0 euros. I wonder how many there are...

In [32]:
print('The number of vehicles in the dataframe that are worth 0 euros is', len(main_df[main_df['price'] == 0]))

The number of vehicles in the dataframe that are worth 0 euros is 10017


It makes sense that some of the vehicles are not worth anything because they are probably in incredibly poor condition and/or otherwise undesirable.

In the event that a customer's car is worth 0 euros, the best the customer can do is donate the car for scrap parts.

In [33]:
main_df['registration_month'].value_counts().sort_index()

0     35096
1     22991
2     21190
3     34223
4     29132
5     29017
6     31323
7     27071
8     22494
9     23709
10    25983
11    24065
12    24150
Name: registration_month, dtype: int64

I find it intriguing that there are not 12, but 13 distinct month values. I will leave this column alone because there is probably a good reason for this that I am unaware of. For example, maybe 1 refers to January, 2 refers to February, 3 refers to March, etc. whereas 0 means "the registration month value is unknown".

In [34]:
main_df['registration_year'].describe()

count    350444.000000
mean       2003.727554
std          61.510710
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64

This column has at least two cars with blatantly incorrect registration year values. At least one car was apparently registered in the year 1000, a year in which cars did not exist yet, and at least one car was apparently registered in the year 9999, which has not happened yet. How many strange registration year values does this dataframe have?

I am completing this project in December 2023, and I wonder which year is the most recent registration year that Rusty Bargain has in its dataframe.

In [35]:
print('The dataframe has...')
print(len(main_df[main_df['registration_year'] == 2016]), 'cars with registration year 2016,')
print(len(main_df[main_df['registration_year'] == 2017]), 'cars with registration year 2017,')
print(len(main_df[main_df['registration_year'] == 2018]), 'cars with registration year 2018,')
print(len(main_df[main_df['registration_year'] == 2019]), 'cars with registration year 2019,')
print(len(main_df[main_df['registration_year'] == 2020]), 'cars with registration year 2020,')
print(len(main_df[main_df['registration_year'] == 2021]), 'cars with registration year 2021,')
print(len(main_df[main_df['registration_year'] == 2022]), 'cars with registration year 2022,')
print('and', len(main_df[main_df['registration_year'] == 2023]), 'cars with registration year 2023.')

The dataframe has...
9297 cars with registration year 2016,
10119 cars with registration year 2017,
3812 cars with registration year 2018,
19 cars with registration year 2019,
0 cars with registration year 2020,
0 cars with registration year 2021,
0 cars with registration year 2022,
and 0 cars with registration year 2023.


Hence, the most recent registration year present in the dataframe is 2019.

In [36]:
print('The number of cars in the dataframe that somehow have a registration year of 2024 or later is', 
      len(main_df[main_df['registration_year'] >= 2024]))

The number of cars in the dataframe that somehow have a registration year of 2024 or later is 66


It would be time-consuming and difficult to find the correct registration year of these 66 vehicles. Thankfully, 66 is a small number compared to the total number of vehicles in the dataframe, so I can drop those 66 rows without harming my models.

In [37]:
main_df = main_df[main_df['registration_year'] < 2024]

As for registration year values that are clearly too old, according to [THIS](https://simple.wikipedia.org/wiki/History_of_the_automobile) Wikipedia article, it was in 1908 that Henry Ford began producing cars that even commoners could afford to drive. Therefore, no car with a registration year earlier than 1908 should be kept in the dataframe.

In [38]:
print('The number of cars in the dataframe that somehow have a registration year of 1907 or earlier is', 
      len(main_df[main_df['registration_year'] <= 1907]))

The number of cars in the dataframe that somehow have a registration year of 1907 or earlier is 38


Thankfully, 38 is a small number compared to the total number of vehicles in the dataframe, so I can drop those 38 rows without harming my model.

In [39]:
main_df = main_df[main_df['registration_year'] > 1907]

In [40]:
main_df['vehicle_type'].value_counts()

sedan          91336
small          79526
wagon          65027
unknown        34397
bus            28734
convertible    20156
coupe          16134
suv            11976
other           3054
Name: vehicle_type, dtype: int64

I see nothing unusual with this column. It does not surprised me that some vehicle types are (much) more common than others.

**Encoding the Dataframe** <a id=3.3></a>

In [41]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350340 entries, 0 to 354106
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   brand               350340 non-null  object        
 1   date_crawled        350340 non-null  datetime64[ns]
 2   date_created        350340 non-null  datetime64[ns]
 3   fuel_type           350340 non-null  object        
 4   gear_box            350340 non-null  object        
 5   horsepower          350340 non-null  float64       
 6   last_seen           350340 non-null  datetime64[ns]
 7   mileage             350340 non-null  int64         
 8   model               350340 non-null  object        
 9   is_repaired         350340 non-null  object        
 10  postal_code         350340 non-null  int64         
 11  registration_month  350340 non-null  int64         
 12  registration_year   350340 non-null  int64         
 13  vehicle_type        350340 no

Due to the preprocessing I did, the dataframe has gone from having 354,369 rows to 350,340 which, thankfully, is a less than 1.5% decrease. I still have an ample amount of data to build accurate machine learning models with.

For some reason, the **horsepower** column now has Dtype float64. The next line converts the column back to Dtype int64.

In [42]:
main_df['horsepower'] = main_df['horsepower'].astype(int)

I will now make a new dataframe called **main_df_encoded**, which is an encoded version of **main_df**. As mentioned earlier, I will not use the columns **date_crawled**, **date_created**, **last_seen**, and **postal_code** to train the models, so they will not be included in **main_df_encoded**. 

Furthermore, I will use an **OrdinalEncoder** on all of the categorical features because machine learning models cannot handle categorical values such as "bmw", "gasoline", "automatic", "golf", "yes", and "sedan".

In [43]:
categorical_features = ['brand', 'fuel_type', 'gear_box', 'model', 'is_repaired', 'vehicle_type']

main_df_encoded = main_df.drop(['date_crawled', 'date_created', 'last_seen', 'postal_code'], axis = 1).copy()

categorical_encoder = OrdinalEncoder()

main_df_encoded[categorical_features] = categorical_encoder.fit_transform(main_df_encoded[categorical_features])

display(main_df_encoded.sample(10))

,brand,fuel_type,gear_box,horsepower,mileage,model,is_repaired,registration_month,registration_year,vehicle_type,price
320807,10.0,6.0,1.0,60,150000,102.0,1.0,1,1997,4.0,250
49986,38.0,2.0,1.0,102,150000,223.0,2.0,5,2006,0.0,8490
77271,27.0,6.0,1.0,75,150000,75.0,2.0,4,2002,5.0,1499
2669,38.0,7.0,1.0,75,150000,116.0,0.0,0,1995,7.0,150
216156,38.0,2.0,1.0,143,150000,203.0,2.0,12,2006,0.0,7750
332211,38.0,7.0,2.0,50,150000,143.0,1.0,5,2005,7.0,999
193588,27.0,6.0,1.0,140,150000,149.0,1.0,0,2005,1.0,3800
102201,38.0,6.0,2.0,60,125000,173.0,1.0,7,1999,5.0,1600
59284,38.0,2.0,1.0,105,150000,116.0,2.0,1,2007,4.0,6900
87032,24.0,6.0,1.0,75,150000,42.0,2.0,1,2000,8.0,1200


Now that the features are all quantitative, I will also use a **MaxAbsScaler** on the features (but not the target, **price**) to scale their values so that no one feature dominates the others. (In particular, the **mileage** column would dominate if I did not do this since those values are, on average, especially large.) However, this should be done after I split the data into training, validation, and test sets.

# Model Training <a id=4></a>

I will split the dataframe into three sets: training, validation, and test, using the commonly used ratio 60:20:20.

In [44]:
_, test_set = train_test_split(main_df_encoded, test_size = 0.2, random_state = 12345)

features_test = test_set.copy().drop(columns = 'price')
target_test = test_set['price'].copy()

In [45]:
main_df_encoded_not_test = main_df_encoded.copy().drop(features_test.index)

training_set, validation_set = train_test_split(main_df_encoded_not_test, test_size = 0.25, random_state = 12345)

features_train = training_set.copy().drop(columns = 'price')
target_train = training_set['price'].copy()

features_valid = validation_set.copy().drop(columns = 'price')
target_valid = validation_set['price'].copy()

Now I can use a **MaxAbsScaler** on the quantitative features of the **features_train** and **features_test** dataframes.

In [46]:
features_col = ['brand', 'fuel_type', 'gear_box', 'horsepower', 'mileage', 'model', 
                'is_repaired', 'registration_month', 'registration_year', 'vehicle_type']

quantitative_encoder = MaxAbsScaler().fit(features_train[features_col].to_numpy())

features_train.loc[:, features_col] = quantitative_encoder.transform(features_train[features_col].to_numpy())
features_valid.loc[:, features_col] = quantitative_encoder.transform(features_valid[features_col].to_numpy())
features_test.loc[:, features_col] = quantitative_encoder.transform(features_test[features_col].to_numpy())

Now I am ready to make my linear regression, decision tree, and random forest models. 

I mentioned earlier that linear regression is not very good for hyperparameter tuning, so my linear regression model will serve as my dummy model, thereby giving me a convenient way to perform a sanity check on my other models.

**Linear Regression Model** <a id=4.1></a>

In [47]:
%%time

lr_model = LinearRegression()
lr_model.fit(features_train, target_train)

CPU times: user 62 ms, sys: 20.3 ms, total: 82.3 ms
Wall time: 92.3 ms


LinearRegression()

In [48]:
%%time

valid_pred_lr_model = lr_model.predict(features_valid)
valid_rmse_lr_model = mean_squared_error(target_valid, valid_pred_lr_model) ** 0.5

CPU times: user 5.13 ms, sys: 20.2 ms, total: 25.3 ms
Wall time: 8.49 ms


In [49]:
print('The root mean square error of the validation set using the linear regression model is', int(valid_rmse_lr_model))

The root mean square error of the validation set using the linear regression model is 3015


The root mean square error in predicting the prices is a little more than 3,000 euros, which I worry is a rather big error. On the bright side, it is great to see that the validation set and the test set acquired very similar results! The similar results imply that the splits were done correctly.

**Decision Tree Model, No Hyperparameter Tuning** <a id=4.2></a>

In [50]:
%%time

dt_model_no_ht = DecisionTreeRegressor(random_state = 12345)
dt_model_no_ht.fit(features_train, target_train)

CPU times: user 1.34 s, sys: 27.9 ms, total: 1.37 s
Wall time: 1.41 s


DecisionTreeRegressor(random_state=12345)

In [51]:
%%time

valid_pred_dt_model_no_ht = dt_model_no_ht.predict(features_valid)
valid_rmse_dt_model_no_ht = mean_squared_error(target_valid, valid_pred_dt_model_no_ht) ** 0.5

CPU times: user 43.1 ms, sys: 0 ns, total: 43.1 ms
Wall time: 41.3 ms


In [52]:
print('The root mean square error of the validation set using the DEFAULT decision tree model is', 
      int(valid_rmse_dt_model_no_ht))

The root mean square error of the validation set using the DEFAULT decision tree model is 2304


Once again, the validation set and the test set acquired very similar results. Furthermore, as expected, the decision tree model is significantly better than the linear regression. More specifically, the RMSE of the decision tree model is close to 25% smaller, meaning it more accurately predicts the prices of used cars than the linear regression. Let's see if hyperparameter tuning can significantly decrease the RMSE even more...

**Decision Tree Model, With Hyperparameter Tuning** <a id=4.3></a>

On the next line is code that uses a RandomizedSearchCV to help me find hyperparameter values for the decision tree model that are better than using the default values. I deliberately have the code commented out because running it would slow down the Jupyter notebook. That said, thanks to a combination of using the RandomizedSearchCV and trial & error I found good hyperparameter values.

In [53]:
#hyperparameters_dt_model = {'max_depth': randint(1, 31),
#                            'min_samples_split': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
#                            'min_samples_leaf': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]}

#grid_search_dt_model = RandomizedSearchCV(estimator = DecisionTreeRegressor(), 
#                                          param_distributions = hyperparameters_dt_model,
#                                          scoring = 'neg_root_mean_squared_error', 
#                                          cv = 3, n_iter = 50)

#grid_search_dt_model.fit(features_train, target_train)

#print(grid_search_dt_model.best_params_)

In [54]:
%%time

dt_model_tuned = DecisionTreeRegressor(random_state = 12345, max_depth = 20, min_samples_leaf = 10, min_samples_split = 4)
dt_model_tuned.fit(features_train, target_train)

CPU times: user 988 ms, sys: 3.75 ms, total: 992 ms
Wall time: 996 ms


DecisionTreeRegressor(max_depth=20, min_samples_leaf=10, min_samples_split=4,
                      random_state=12345)

In [55]:
%%time

valid_pred_dt_model_tuned = dt_model_tuned.predict(features_valid)
valid_rmse_dt_model_tuned = mean_squared_error(target_valid, valid_pred_dt_model_tuned) ** 0.5

CPU times: user 19 ms, sys: 3.8 ms, total: 22.8 ms
Wall time: 23.9 ms


In [56]:
print('The root mean square error of the validation set using the TUNED decision tree model is', 
      int(valid_rmse_dt_model_tuned))

The root mean square error of the validation set using the TUNED decision tree model is 1975


By setting **max_depth** = 20, **min_samples_leaf** = 10, and **min_samples_split** = 4 the RMSE has been reduced by more than 300, a significant improvement for sure! Furthermore, the wall time has been reduced as well! I wonder how a random forest will do...

**Random Forest Model, No Hyperparameter Tuning** <a id=4.4></a>

In [57]:
%%time

rf_model_no_ht = RandomForestRegressor(random_state = 12345)
rf_model_no_ht.fit(features_train, target_train)

CPU times: user 1min 27s, sys: 560 ms, total: 1min 28s
Wall time: 1min 28s


RandomForestRegressor(random_state=12345)

In [58]:
%%time

valid_pred_rf_model_no_ht = rf_model_no_ht.predict(features_valid)
valid_rmse_rf_model_no_ht = mean_squared_error(target_valid, valid_pred_rf_model_no_ht) ** 0.5

CPU times: user 3.56 s, sys: 7.94 ms, total: 3.57 s
Wall time: 3.59 s


In [59]:
print('The root mean square error of the validation set using the DEFAULT random forest model is', 
      int(valid_rmse_rf_model_no_ht))

The root mean square error of the validation set using the DEFAULT random forest model is 1771


On the bright side, the RMSE of the random forest model is even smaller than the decision tree model, and a little more than 40% lower compared to the linear regression. However, this model takes <b><u>FAR</b></u> longer than the decision tree to process. The wall time of the decision tree is less than 2 seconds, whereas the wall time of the random forest is more than 1 <b><i>minute</b></i>! I wonder if hyperparameter tuning can decrease both the RMSE (even more) and the wall time...

**Random Forest Model, With Hyperparameter Tuning** <a id=4.5></a>

Just like with the decision tree model, the next line is code uses a RandomizedSearchCV to help me find hyperparameter values for the random forest model that are better than using the default values. Once more, I deliberately have the code commented out because running it would slow down the Jupyter notebook. After acquiring the best hyperparameter values according to RandomizedSearchCV, I once again did some trial & error to tweak the model a bit.

In [60]:
#hyperparameters_rf_model = {'max_depth': randint(1, 21),
#                            'min_samples_split': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
#                            'min_samples_leaf': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
#                            'n_estimators': randint(1, 31)}

#grid_search_rf_model = RandomizedSearchCV(estimator = RandomForestRegressor(), 
#                                          param_distributions = hyperparameters_rf_model,
#                                          scoring = 'neg_root_mean_squared_error', 
#                                          cv = 3, n_iter = 10)

#grid_search_rf_model.fit(features_train, target_train)

#print(grid_search_rf_model.best_params_)

In [61]:
%%time

rf_model_tuned = RandomForestRegressor(random_state = 12345, max_depth = 20, n_estimators = 25,
                                       min_samples_leaf = 2, min_samples_split = 8)

rf_model_tuned.fit(features_train, target_train)

CPU times: user 17.7 s, sys: 39.8 ms, total: 17.8 s
Wall time: 17.8 s


RandomForestRegressor(max_depth=20, min_samples_leaf=2, min_samples_split=8,
                      n_estimators=25, random_state=12345)

In [62]:
%%time

valid_pred_rf_model_tuned = rf_model_tuned.predict(features_valid)
valid_rmse_rf_model_tuned = mean_squared_error(target_valid, valid_pred_rf_model_tuned) ** 0.5

CPU times: user 492 ms, sys: 0 ns, total: 492 ms
Wall time: 503 ms


In [63]:
print('The root mean square error of the validation set using the TUNED random forest model is', 
      int(valid_rmse_rf_model_tuned))

The root mean square error of the validation set using the TUNED random forest model is 1767


I am honestly very surprised to see that the RMSE values of the tuned random forest model are only a tiny bit better than those of the default random forest. That said, the wall time is now **MUCH** shorter, about 20 seconds instead of over 1 minute.

Unfortunately, I still don't think that this improved wall time is particularly good. Thankfully, I recently learned about three other models that I can test out, whose names are CatBoostRegressor, LGBMRegressor, and XGBRegressor. If any of those three acquire RMSE values that are even smaller than those of the tuned random forest while also having a smaller wall time than the tuned random forest, that would be ideal.

**CatBoost Regressor, No Hyperparameter Tuning** <a id=4.6></a>

In [64]:
%%time

cb_model_no_ht = CatBoostRegressor(random_state = 12345)
cb_model_no_ht.fit(features_train, target_train, verbose = 100)

Learning rate set to 0.095314
0:	learn: 4253.3917925	total: 100ms	remaining: 1m 39s
100:	learn: 1923.2443937	total: 4.6s	remaining: 41s
200:	learn: 1835.3314705	total: 9.1s	remaining: 36.2s
300:	learn: 1787.6560480	total: 13.6s	remaining: 31.6s
400:	learn: 1756.3402414	total: 18.1s	remaining: 27.1s
500:	learn: 1731.9435852	total: 22.7s	remaining: 22.6s
600:	learn: 1711.4408302	total: 27.1s	remaining: 18s
700:	learn: 1694.3209699	total: 31.6s	remaining: 13.5s
800:	learn: 1679.4697128	total: 36.1s	remaining: 8.96s
900:	learn: 1665.9959758	total: 40.5s	remaining: 4.45s
999:	learn: 1654.3790547	total: 44.9s	remaining: 0us
CPU times: user 45 s, sys: 51.5 ms, total: 45.1 s
Wall time: 45.3 s


In [65]:
%%time

valid_pred_cb_model_no_ht = cb_model_no_ht.predict(features_valid)
valid_rmse_cb_model_no_ht = mean_squared_error(target_valid, valid_pred_cb_model_no_ht) ** 0.5

CPU times: user 90.9 ms, sys: 26 µs, total: 90.9 ms
Wall time: 89.2 ms


In [66]:
print('The root mean square error of the validation set using the DEFAULT CatBoost regressor is', 
      int(valid_rmse_cb_model_no_ht))

The root mean square error of the validation set using the DEFAULT CatBoost regressor is 1768


These RMSE values are extremely similar to those of the random forest models. Let's see if hyperparameter tuning changes this.

**CatBoost Regressor, With Hyperparameter Tuning** <a id=4.7></a>

For this model, I use GridSearchCV instead of RandomizedSearchCV because I did not make there be as many possible combinations to go through, so it would not be overly time-consuming to check all of them and find out which is the best one.

In [67]:
#hyperparameters_cb_model = {'depth': [5, 10],
#                            'iterations': [100, 150, 200, 250],
#                            'learning_rate': [0.1, 0.2]}


#grid_search_cb_model = GridSearchCV(estimator = CatBoostRegressor(), 
#                                    param_grid = hyperparameters_cb_model,
#                                    scoring = 'neg_root_mean_squared_error', cv = 3)

#grid_search_cb_model.fit(features_train, target_train, verbose = 100)

#print(grid_search_cb_model.best_params_)

In [68]:
%%time

cb_model_tuned = CatBoostRegressor(random_state = 12345, depth = 10, iterations = 250, learning_rate = 0.2)
cb_model_tuned.fit(features_train, target_train, verbose = 50)

0:	learn: 3901.9834751	total: 140ms	remaining: 34.8s
50:	learn: 1776.5429646	total: 6.82s	remaining: 26.6s
100:	learn: 1677.8302034	total: 13.4s	remaining: 19.7s
150:	learn: 1610.6310353	total: 19.9s	remaining: 13s
200:	learn: 1565.8076164	total: 26.4s	remaining: 6.44s
249:	learn: 1526.8097073	total: 32.8s	remaining: 0us
CPU times: user 32.8 s, sys: 63.8 ms, total: 32.8 s
Wall time: 33.2 s


In [69]:
%%time

valid_pred_cb_model_tuned = cb_model_tuned.predict(features_valid)
valid_rmse_cb_model_tuned = mean_squared_error(target_valid, valid_pred_cb_model_tuned) ** 0.5

CPU times: user 84.8 ms, sys: 3.98 ms, total: 88.8 ms
Wall time: 98.9 ms


In [70]:
print('The root mean square error of the validation set using the TUNED CatBoost regressor is', 
      int(valid_rmse_cb_model_tuned))

The root mean square error of the validation set using the TUNED CatBoost regressor is 1739


Tuning the CatBoost regressor made the RMSE values a little better, and the wall time is a little reduced. The tuned CatBoost regressor has RMSE values that are slightly better than the tuned random forest, but the wall time is over 10 seconds longer.

**LightGBM Regressor, No Hyperparameter Tuning** <a id=4.8></a>

In [71]:
%%time

lgbm_model_no_ht = LGBMRegressor(random_state = 12345)
lgbm_model_no_ht.fit(features_train, target_train)

CPU times: user 3.39 s, sys: 40 ms, total: 3.43 s
Wall time: 3.4 s


LGBMRegressor(random_state=12345)

In [72]:
%%time

valid_pred_lgbm_model_no_ht = lgbm_model_no_ht.predict(features_valid)
valid_rmse_lgbm_model_no_ht = mean_squared_error(target_valid, valid_pred_lgbm_model_no_ht) ** 0.5

CPU times: user 641 ms, sys: 4 µs, total: 641 ms
Wall time: 615 ms


In [73]:
print('The root mean square error of the validation set using the DEFAULT LightGBM regressor is', 
      int(valid_rmse_lgbm_model_no_ht))

The root mean square error of the validation set using the DEFAULT LightGBM regressor is 1851


These RMSE values are in between those of the tuned random forest and the tuned decision tree. This default LightGBM model has a noticeably larger wall time value than the tuned decision tree, but thankfully it is much smaller than the wall time of the tuned random forest, which suggests to me that using a tuned LightGBM model has potential to be one that strikes an excellent balance between speed and accuracy.

**LightGBM Regressor, With Hyperparameter Tuning** <a id=4.9></a>

In [74]:
#hyperparameters_lgbm_model = {'max_depth': [5, 10],
#                              'n_estimators': [50, 100, 150, 200],
#                              'learning_rate': [0.1, 0.2]}


#grid_search_lgbm_model = GridSearchCV(estimator = LGBMRegressor(), 
#                                      param_grid = hyperparameters_lgbm_model,
#                                      scoring = 'neg_root_mean_squared_error', cv = 3)

#grid_search_lgbm_model.fit(features_train, target_train)

#print(grid_search_lgbm_model.best_params_)

In [75]:
%%time

lgbm_model_tuned = LGBMRegressor(random_state = 12345, max_depth = 10, n_estimators = 200, learning_rate = 0.2)
lgbm_model_tuned.fit(features_train, target_train)

CPU times: user 4.75 s, sys: 43.8 ms, total: 4.8 s
Wall time: 4.87 s


LGBMRegressor(learning_rate=0.2, max_depth=10, n_estimators=200,
              random_state=12345)

In [76]:
%%time

valid_pred_lgbm_model_tuned = lgbm_model_tuned.predict(features_valid)
valid_rmse_lgbm_model_tuned = mean_squared_error(target_valid, valid_pred_lgbm_model_tuned) ** 0.5

CPU times: user 968 ms, sys: 7.9 ms, total: 976 ms
Wall time: 998 ms


In [77]:
print('The root mean square error of the validation set using the TUNED LightGBM regressor is', 
      int(valid_rmse_lgbm_model_tuned))

The root mean square error of the validation set using the TUNED LightGBM regressor is 1764


This is a great result! The tuned LightGBM regressor has RMSE values that are slightly better than the tuned random forest, and its wall time is less than half that of the tuned random forest! More specifically, the tuned LightGBM regressor has a wall time greater than 5 seconds, but less than 10. Though the wall time of the tuned LightGBM regressor is noticeably longer than that of the tuned decision tree, I personally think the additional time is worth it because the tuned LightGBM regressor has RMSE values that are a little more than 200 (euros) smaller.

**XGB Regressor, No Hyperparameter Tuning** <a id=4.10></a>

In [78]:
%%time

xgbr_model_no_ht = XGBRegressor(random_state = 12345)
xgbr_model_no_ht.fit(features_train, target_train)

CPU times: user 30.1 s, sys: 56.6 ms, total: 30.1 s
Wall time: 30.3 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=12345,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [79]:
%%time

valid_pred_xgbr_model_no_ht = xgbr_model_no_ht.predict(features_valid)
valid_rmse_xgbr_model_no_ht = mean_squared_error(target_valid, valid_pred_xgbr_model_no_ht) ** 0.5

CPU times: user 354 ms, sys: 3.99 ms, total: 358 ms
Wall time: 362 ms


In [80]:
print('The root mean square error of the validation set using the DEFAULT XGB regressor is', 
      int(valid_rmse_xgbr_model_no_ht))

The root mean square error of the validation set using the DEFAULT XGB regressor is 1781


I am not fond of the fact that the wall time is about 30 seconds, which is longer than the tuned random forest despite having similar RMSE values. With that said, the decision tree model (in particular) was greatly improved thanks to tuning the hyperparameters, and if that is the case for the XGB regressor as well, then it might be even better to use than the tuned LightGBM regressor.

**XGB Regressor, With Hyperparameter Tuning** <a id=4.11></a>

In [81]:
#hyperparameters_xgbr_model = {'max_depth': [5, 10],
#                              'n_estimators': [25, 50, 75, 100],
#                              'learning_rate': [0.1, 0.2]}


#grid_search_xgbr_model = GridSearchCV(estimator = XGBRegressor(), 
#                                      param_grid = hyperparameters_xgbr_model,
#                                      scoring = 'neg_root_mean_squared_error', cv = 3)

#grid_search_xgbr_model.fit(features_train, target_train)

#print(grid_search_xgbr_model.best_params_)

In [82]:
%%time

xgbr_model_tuned = XGBRegressor(random_state = 12345, max_depth = 10, n_estimators = 100, learning_rate = 0.2)
xgbr_model_tuned.fit(features_train, target_train)

CPU times: user 58.4 s, sys: 87.9 ms, total: 58.5 s
Wall time: 59.1 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.2, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=12345,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [83]:
%%time

valid_pred_xgbr_model_tuned = xgbr_model_tuned.predict(features_valid)
valid_rmse_xgbr_model_tuned = mean_squared_error(target_valid, valid_pred_xgbr_model_tuned) ** 0.5

CPU times: user 692 ms, sys: 51 µs, total: 692 ms
Wall time: 682 ms


In [84]:
print('The root mean square error of the validation set using the TUNED XGB regressor is', 
      int(valid_rmse_xgbr_model_tuned))

The root mean square error of the validation set using the TUNED XGB regressor is 1713


The XGBR model is not worth using. Though I managed to trim down the RMSE values a bit (by less than 100), using hyperparameter tuning is not worth it with this model because it causes the wall time to approximately double. I would not want to use the default XGBR model either because I see no advantages to using it over the tuned decision tree, the tuned random forest, and the tuned LightGBM regressor models.

# Model Analysis <a id=5></a>

Let's store the above RMSE values into a new dataframe. Furthermore, I would also like for the dataframe to indicate whether or not the wall time is less than 30 seconds. My reasoning for this is, from my perspective, a wall time greater than 30 seconds is too slow. A wall time less than 10 seconds would be ideal, but a wall time between 10 seconds and 30 seconds is acceptable as well.

In [85]:
summary_df = pd.DataFrame()

summary_df['model_name'] = ['linear_reg', 'dt_default', 'dt_tuned', 'rf_default', 'rf_tuned','cat_default', 
                            'cat_tuned', 'light_default', 'light_tuned', 'xgbr_default', 'xgbr_tuned']

In [86]:
summary_df['time_under_30'] = ['Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No']

In [87]:
summary_df['rmse'] = [int(valid_rmse_lr_model), int(valid_rmse_dt_model_no_ht), int(valid_rmse_dt_model_tuned),
                      int(valid_rmse_rf_model_no_ht), int(valid_rmse_rf_model_tuned), int(valid_rmse_cb_model_no_ht), 
                      int(valid_rmse_cb_model_tuned), int(valid_rmse_lgbm_model_no_ht), int(valid_rmse_lgbm_model_tuned), 
                      int(valid_rmse_xgbr_model_no_ht), int(valid_rmse_xgbr_model_tuned)]

In my opinion, any model whose wall time is more than 30 seconds should not even be considered because there are multiple models whose wall time is under 30 seconds **AND** that have relatively low RMSE values. 

In [88]:
summary_df_under_30 = summary_df[summary_df['time_under_30'] == 'Yes'].copy()
summary_df_under_30 = summary_df_under_30.drop('time_under_30', axis = 1)

Recall that the linear regression is being used as the dummy model. Hence, I think it is appropriate to add a column to this dataframe that tells how much less the average RMSE value of each model is compared to that of the linear regression.

In [89]:
summary_df_under_30['rmse_minus_lr_rmse'] = summary_df_under_30['rmse'] - int(valid_rmse_lr_model)

Now let's add a column that expresses how much less the average RMSE value of each model is compared to that of the linear regression as a <u>percentage</u>.

In [90]:
summary_df_under_30['perc_dec_from_lr'] = round((summary_df_under_30['rmse'] - int(valid_rmse_lr_model)) 
                                                / int(valid_rmse_lr_model) * 100, 1)

In [91]:
display(summary_df_under_30)

,model_name,rmse,rmse_minus_lr_rmse,perc_dec_from_lr
0,linear_reg,3015,0,0.0
1,dt_default,2304,-711,-23.6
2,dt_tuned,1975,-1040,-34.5
4,rf_tuned,1767,-1248,-41.4
7,light_default,1851,-1164,-38.6
8,light_tuned,1764,-1251,-41.5


Thanks to this dataframe, I am even more convinced that the tuned LightGBM Regressor is the best one to use. That model is relatively fast, and its RMSE value is the biggest decrease from the linear regression's RMSE value. 

One last thing I would like to do is assess the tuned LightGBM Regressor using the test set.

In [92]:
test_pred_lgbm_model_tuned = lgbm_model_tuned.predict(features_test)
test_rmse_lgbm_model_tuned = mean_squared_error(target_test, test_pred_lgbm_model_tuned) ** 0.5

print('The root mean square error of the test set using the TUNED LightGBM regressor is', int(test_rmse_lgbm_model_tuned))

The root mean square error of the test set using the TUNED LightGBM regressor is 1774


Excellent! This RMSE value is very similar to the value I acquired using the validation set, which is exactly what should happen if the model is works correctly.

# Conclusion <a id=6></a>

Rusty Bargain hopes to attract new customers using an app that is intended to quickly determine the market value of a prospective customer's used car. 

My responsibility was to build a model for the app that predicts the prices of used cars quickly and accurately, while also making sure that the model does not take too long to train.

To help me build such a model, I tested each of the following regressors, both with and without hyperparameter tuning: decision tree, random forest, CatBoost, LightGBM, and and XGBR.

I narrowed down the options to the models where the combined wall time between training the model and making predictions is less than 30 seconds, and from there looked to see which one had the smallest root mean square error, based on the results of using the validation set on the models.

I confidently conclude that the hyperparameter tuned LightGBM model is the best one to use, as it has a combined wall time of under 10 seconds, and the smallest RMSE value. The tuned Decision Tree model is faster, but less accurate, and the tuned Random Forest model is approximately as accurate, but more than two times slower. The tuned LightGBM model strikes the best balance between speed and accuracy.